We implement a matrix factorisation method, SVD (singular value decomposition) here to solve this problem.

## Recommender System using SVD

Matrix factorization can be used to discover features underlying the interactions between two different kinds of entities. And one obvious application is to predict ratings in collaborative filtering—in other words, to recommend items to users.

One advantage of employing matrix factorization for recommender systems is the fact that it can incorporate implicit feedback—information that’s not directly given but can be derived by analyzing user behavior—such as items frequently bought or viewed.

Using this capability we can estimate if a user is going to like a movie that they never saw. And if that estimated rating is high, we can recommend that movie to the user, so as to provide a more personalized experience.

For example, two users might give high ratings to a certain movie if they both like the actors/actresses of the movie, or if the movie is a thriller movie, which is a genre preferred by both users.

Hence, if we can discover these kinds of latent features (like genre or actors and directors), we should be able to predict a rating with respect to a certain user and a certain item, because the features associated with the user should match with the features associated with the item.


The data input for a recommender system can be thought of as a large matrix, with the rows indicating an entry for a customer, and the columns indicating an entry for a particular item. Let’s call this matrix $R$. Then entry $R_{i,j}$ will contain the score that customer $i$ has given to product $j$. For example if it’s a review this could be a number from 1–5, or it might just be 0–1 indicating if a user has bought an item or not. This matrix contains a lot of missing information, it’s unlikely a customer has bought every item on Amazon! Recommender systems aim to fill in this missing information, by predicting the customer score of items where the score is missing. Then recommender systems will recommend items to the customer that have the highest score. A typical example of the matrix $R$ with entries that are review values from 1–5 is given in the picture below.

<img src="images/ex_im_1.png" alt="Utility Matrix" style="width: 80%"/>

#### Imports

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy.linalg import sqrtm

#### Dataset

The dataset we’ll be working with is a very famous movies dataset: the ml-1m, or the [MovieLens dataset 100 k](https://grouplens.org/datasets/movielens/100k).

##### 1. Data Preprocessing
We will begin by loading the dataset file present in the `.csv` file into pandas dataframes and visualizing the entries.

In [2]:
data = pd.read_csv('movielens100k.csv')
data['userId'] = data['userId'].astype('str')
data['movieId'] = data['movieId'].astype('str')

users = data['userId'].unique() #list of all users
movies = data['movieId'].unique() #list of all movies

print("Number of users", len(users))
print("Number of movies", len(movies))

data.head()

Number of users 718
Number of movies 8915


,userId,movieId,rating,timestamp
0,1,1,5.0,847117005
1,1,2,3.0,847642142
2,1,10,3.0,847641896
3,1,32,4.0,847642008
4,1,34,4.0,847641956


##### 2. Split the data into a train and test set

In [3]:
test = pd.DataFrame(columns=data.columns)
train = pd.DataFrame(columns=data.columns)
test_ratio = 0.2 #fraction of data to be used as test set.
for u in users:
    temp = data[data['userId'] == u]
    n = len(temp)
    test_size = int(test_ratio*n)
    temp = temp.sort_values('timestamp').reset_index()
    temp.drop('index', axis=1, inplace=True)
        
    dummy_test = temp.iloc[n-1-test_size :]
    dummy_train = temp.iloc[: n-2-test_size]
        
    test = pd.concat([test, dummy_test])
    train = pd.concat([train, dummy_train])

##### 3. Create the utility matrix

The input data will now be converted to the utility matrix $(n\times m)$ where the rows of the matrix are users $n$ and the columns are the ratings for the $m$-th movie.

In [4]:
def create_utility_matrix(data, formatizer = {'user':0, 'item': 1, 'value': 2}):
    """
        :param data:      Array-like, 2D, nx3
        :param formatizer:pass the formatizer
        :return:          utility matrix (n x m), n=users, m=items
    """
        
    itemField = formatizer['item']
    userField = formatizer['user']
    valueField = formatizer['value']
    userList = data.iloc[:,userField].tolist()
    itemList = data.iloc[:,itemField].tolist()
    valueList = data.iloc[:,valueField].tolist()
    users = list(set(data.iloc[:,userField]))
    items = list(set(data.iloc[:,itemField]))
    users_index = {users[i]: i for i in range(len(users))}
    pd_dict = {item: [np.nan for i in range(len(users))] for item in items}
    for i in range(0,len(data)):
        item = itemList[i]
        user = userList[i]
        value = valueList[i]
        pd_dict[item][users_index[user]] = value
    X = pd.DataFrame(pd_dict)
    X.index = users
        
    itemcols = list(X.columns)
    items_index = {itemcols[i]: i for i in range(len(itemcols))}
    # users_index gives us a mapping of user_id to index of user
    # items_index provides the same for items
    return X, users_index, items_index

utilMat, users_index, items_index = create_utility_matrix(train)

#### Metric computation

The function rmse computes the root mean square error (RMSE) for the true and the predicted movie ratings.

In [5]:
def rmse(true, pred):
    # this will be used to compute the root mean square error for the true and the predicted movie rating
    x = true - pred
    return sum([xi*xi for xi in x])/len(x)

#### Code for computing SVD for the utility matrix [Write your code here]

In [6]:
def svd(train, k):
    utilMat = np.array(train)

    mask = np.isnan(utilMat)
    masked_arr=np.ma.masked_array(utilMat, mask)
    item_means=np.mean(masked_arr, axis=0)
    utilMat = masked_arr.filled(item_means)

    x = np.tile(item_means, (utilMat.shape[0],1))

    utilMat = utilMat - x

    U, s, V=np.linalg.svd(utilMat, full_matrices=False)
    s=np.diag(s)
    s=s[0:k,0:k]
    U=U[:,0:k]
    V=V[0:k,:]

    s_root=sqrtm(s)

    Usk=np.dot(U,s_root)
    skV=np.dot(s_root,V)
    UsV = np.dot(Usk, skV)

    UsV = UsV + x

    print("svd computed")
    return UsV

#### Code for the test set [Write your code here]

Write the code that computes the RMSE for the predicted ratings for the test data present in the `test` matrix.

In [7]:
no_of_features = [8,10,12,14,17]
#To test the performance over a different number of features

utilMat, users_index, items_index = create_utility_matrix(train)

for f in no_of_features: 
    svdout = svd(utilMat, k=f)
    pred = [] 
#to store the predicted ratings

    for _,row in test.iterrows():
        user = row['userId']
        item = row['movieId']

        u_index = users_index[user]
        if item in items_index:
            i_index = items_index[item]
            pred_rating = svdout[u_index, i_index]
        else:
            pred_rating = np.mean(svdout[u_index, :])
        pred.append(pred_rating)

    print(rmse(test['rating'], pred))

svd computed
0.963089854348795
svd computed
0.9632320309708745
svd computed
0.9645970994047942
svd computed
0.9632251443983305
svd computed
0.9624754027509489
